### Import libraries

In [1]:

import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas.io.json import json_normalize

import time

from datetime import datetime
from dateutil import tz

from bibliopixel import *
from bibliopixel.drivers.LPD8806 import *
from bibliopixel import LEDStrip
import bibliopixel.colors as colors

ImportError: No module named 'bibliopixel'

In [20]:
## read data
df_total = pd.read_pickle('Lindblom_All_meters_concat')


### Generating color scale

In [4]:
red = 0
green = 255
stepSize = 50
color_gn_rd = []
color_gn_rd.append((red, green, 0)) ## the lights are GRB format

In [5]:
while(red < 255): ## start with green and increase red
    red += stepSize;
    if(red > 245):
        red = 255; 
    color_gn_rd.append((red, green, 0))

while(green > 0): ## start with red + green and decrease green
    green -= stepSize;
    if(green < 6):
        green = 0; 
    color_gn_rd.append((red, green, 0)); 
    
total_colors = len(color_gn_rd)

### Setting up LEDs

In [5]:
LedsPerSide = 10
numLeds= LedsPerSide*4*2 ##x/side * 4 sides * 2 levels
driver=DriverLPD8806(numLeds, ChannelOrder.BRG)
led=LEDStrip(driver)

ERROR - spi_driver_base - Unable to import spidev. Please install. pip install spidev
ERROR:BiblioPixel:Unable to import spidev. Please install. pip install spidev


ImportError: Unable to import spidev. Please install. pip install spidev

### Defining Flashing modes

In [6]:
def led_set(start_position, numLEDs, color): ## Fills the colors
    #led.fill(color, start=start_position,end=start_position+numLEDs)
    #led.update()
    print color
    return

def led_pulse(start_position, numLEDs, color):
    ## Step-up intensity by 10% increments, then step down by the same every 0.1 seconds. total time = 4 sec
    
    intensity = np.arange(0,1.1,0.1) 

    for i in intensity:
        color_new = (int(color[0]*i),int(color[1]*i),int(color[2]*i)) 
        # There is probably a more elegant way to do this.. 
        print color_new
        #led.fill(color_new, start=start_position,end=start_position+numLEDs)
        #led.update()
        time.sleep(0.1)

    for i in reversed(intensity):
        color_new = (int(color[0]*i),int(color[1]*i),int(color[2]*i)) 
        print color_new
        #led.fill(color_new, start=start_position,end=start_position+numLEDs)
        #led.update()
        time.sleep(0.1)        

### Calculate Average Data

In [21]:
df_total

,duration,time_stamp,value,meter ID
0,1800,2006-01-01 07:00:00,8.64,97121730
1,1800,2006-01-01 07:30:00,8.64,97121730
2,1800,2006-01-01 08:00:00,8.82,97121730
3,1800,2006-01-01 08:30:00,8.82,97121730
4,1800,2006-01-01 09:00:00,8.82,97121730
5,1800,2006-01-01 09:30:00,8.28,97121730
6,1800,2006-01-01 10:00:00,8.46,97121730
7,1800,2006-01-01 10:30:00,8.46,97121730
8,1800,2006-01-01 11:00:00,8.10,97121730
9,1800,2006-01-01 11:30:00,8.10,97121730


In [24]:
#Added by Month


df_total['Month'] = pd.DatetimeIndex(df_total['time_stamp']).month
df_total['Year'] = pd.DatetimeIndex(df_total['time_stamp']).year
df_Monthlygroup = df_total.groupby(['Month', 'Year'])
df_MonthlyAverage = df_Monthlygroup['value'].mean()*2 #X2 gives average kwh/h --> kw
df_MonthlyTotal = df_Monthlygroup['value'].sum()
df_MonthlyAverage.to_csv('Dunne_Monthly_Average_KW.csv', sep=',')
df_MonthlyTotal.to_csv('Dunne_Monthly_Total_KWH.csv', sep=',')


#Added by Day

df_total['Date'] = pd.DatetimeIndex(df_total['time_stamp']).date
df_Dailygroup = df_total.groupby('Date')
df_DailyAverage = df_Dailygroup['value'].mean()*2
df_DailyTotal = df_Dailygroup['value'].sum()
df_DailyMax = df_Dailygroup['value'].max()
df_1week_DailyMax = df_DailyMax[(df_new.index > '2015-9-19 00:00:00') & (df_new.index < '2015-9-26 00:00:00')]
df_DailyAverage.to_csv('Dunne_Daily_Average_KW.csv', sep=',')
df_DailyTotal.to_csv('Dunne_Daily_Total_KWH.csv', sep=',')
df_1week_DailyMax = to_csv('Dunne_1week_Daily_Max_KWH.csv', sep=',')

In [42]:
df_MonthlyAverage[1,2015]

56.732834677419511

In [16]:
df_group = df_total.groupby('time_stamp')
df_new = df_group['value'].sum()

df_1Week = df_new[(df_new.index > '2015-9-19 00:00:00') & (df_new.index < '2015-9-26 00:00:00')]
df_1Week
df_1Week.to_csv('df_1Week_Dunne.csv', sep=',')

In [83]:
df_DailyAverage = df_Dailygroup['value'].sum()
df_new.to_csv('df_dailytotal_dunne.csv', sep=',')

df_RollingMean = pd.rolling_mean(df_new, 30)
df_RollingMean
df_new

time_stamp
2015-01-16 01:00:00    54.252
2015-01-16 01:30:00    55.044
2015-01-16 02:00:00    55.908
2015-01-16 02:30:00    55.476
2015-01-16 03:00:00    55.764
2015-01-16 03:30:00    55.683
2015-01-16 04:00:00    56.259
2015-01-16 04:30:00    55.755
2015-01-16 05:00:00    55.611
2015-01-16 05:30:00    56.250
2015-01-16 06:00:00    57.186
2015-01-16 06:30:00    56.331
2015-01-16 07:00:00    57.987
2015-01-16 07:30:00    59.724
2015-01-16 08:00:00    62.784
2015-01-16 08:30:00    61.560
2015-01-16 09:00:00    62.703
2015-01-16 09:30:00    70.839
2015-01-16 10:00:00    67.671
2015-01-16 10:30:00    65.592
2015-01-16 11:00:00    62.712
2015-01-16 11:30:00    59.040
2015-01-16 12:00:00    52.056
2015-01-16 12:30:00    50.247
2015-01-16 13:00:00    49.752
2015-01-16 13:30:00    50.895
2015-01-16 14:00:00    53.559
2015-01-16 14:30:00    51.696
2015-01-16 15:00:00    51.255
2015-01-16 15:30:00    49.662
                        ...  
2016-02-16 04:00:00    32.525
2016-02-16 04:30:00    33.413

### Displaying Daily Total Values for the entire year of data

In [24]:
df_DailyAverage = pd.read_csv('df_DailyAverage_Dunne.csv',  parse_dates=True)
df_RollingMean = pd.read_csv('df_RollingMean_Dunne.csv',  parse_dates=True)
df_1Week = pd.read_csv('df_1Week_Dunne.csv', parse_dates=True)

In [8]:
def yearly_data():
    while (True):
        print 'Starting Display'
        print 'Press \'Control + C\' to stop'
        max_value = df_DailyAverage.max()
        ScalingSteps = (max_value + 0.1)/(total_colors)
        for item in df_DailyAverage.iteritems():
            print 'Date: ', item[0]
            print 'Average use: ', item [1]
            color_index = int(item[1]/ScalingSteps)
            print color_index
            # print color_index
            color = color_gn_rd[color_index]
            led_set(0, 80, color)

### Displaying Daily Energy Usage on Bottom with last 30 days' average on Top

In [9]:
def daily_vs_past30days():
    while (True):
        print 'Starting Display'
        print 'Press \'Control + C\' to stop'
        max_value = df_DailyAverage.max()
        ScalingSteps = (max_value + 0.1)/total_colors
        count = 0
        for item in df_DailyAverage.iteritems():
            print 'Date: ', item[0]
            print 'Total use: ', item [1]
            color_index = int(item[1]/ScalingSteps)

            # print color_index
            color = color_gn_rd[color_index]
            led_set(0, 40, color)

            print 'Last 30 days average: ', df_RollingMean[count]
            try:
                color_index = int(df_RollingMean[count]/ScalingSteps)
                color = color_gn_rd[color_index]
                led_set(40, 80, color)

            except:
                color = (0,0,0)
                led_set(40, 80, color)

            count = count + 1

    

### Displaying 1 Week's Energy Use (res: every half-hour )

In [66]:
def OneWeek_data():
    time_stamp = []
    value = []
    with open('df_1Week_Dunne.csv') as f:
        cf = csv.DictReader(f, fieldnames=['time_stamp', 'value'])
        for row in cf:
            time_stamp.append(row['time_stamp'])
            value.append(float(row['value']))

    days = ['Saturday', 'Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
    max_value = max(value)
    ScalingSteps = (max_value + 0.1)/total_colors
    date = []
    while (True):
        print 'Starting Display'
        print 'Press \'Control + C\' to stop'
        
        count = 0
        for ts, val in zip(time_stamp, value):
            
            
            if(date != time.strftime("%Y-%m-%d",  time.strptime(ts, "%Y-%m-%d %H:%M:%S"))):
                print days[count]
                led_pulse(0, 80, (0,0,255))
                count = count + 1

            print 'Time: ', ts
            print 'Electricity Usage: ', val
            color_index = int((val)/ScalingSteps)

            print color_index
            color = color_gn_rd[color_index]
            led_set(0, 80, color)

            date = time.strftime("%Y-%m-%d",  time.strptime(ts, "%Y-%m-%d %H:%M:%S"))

In [11]:
print 'Functions Available:'
print 'yearly_data()'
print 'daily_vs_past30days()'
print 'OneWeek_data()'

Functions Available:
yearly_data()
daily_vs_past30days()
OneWeek_data()


In [68]:
OneWeek_data()

Starting Display
Press 'Control + C' to stop
Saturday
(0, 0, 0)
(0, 0, 25)
(0, 0, 51)
(0, 0, 76)
(0, 0, 102)
(0, 0, 127)
(0, 0, 153)
(0, 0, 178)
(0, 0, 204)
(0, 0, 229)
(0, 0, 255)
(0, 0, 255)
(0, 0, 229)
(0, 0, 204)
(0, 0, 178)
(0, 0, 153)
(0, 0, 127)
(0, 0, 102)
(0, 0, 76)
(0, 0, 51)
(0, 0, 25)
(0, 0, 0)
Time:  2015-09-19 00:30:00
Electricity Usage:  16.92
5
(255, 255, 0)
Time:  2015-09-19 01:00:00
Electricity Usage:  17.208
5
(255, 255, 0)
Time:  2015-09-19 01:30:00
Electricity Usage:  16.128
5
(255, 255, 0)
Time:  2015-09-19 02:00:00
Electricity Usage:  16.2
5
(255, 255, 0)
Time:  2015-09-19 02:30:00
Electricity Usage:  15.48
5
(255, 255, 0)
Time:  2015-09-19 03:00:00
Electricity Usage:  16.272
5
(255, 255, 0)
Time:  2015-09-19 03:30:00
Electricity Usage:  14.76
4
(200, 255, 0)
Time:  2015-09-19 04:00:00
Electricity Usage:  14.904
4
(200, 255, 0)
Time:  2015-09-19 04:30:00
Electricity Usage:  14.04
4
(200, 255, 0)
Time:  2015-09-19 05:00:00
Electricity Usage:  14.832
4
(200, 255, 0

KeyboardInterrupt: 

In [56]:
time_stamp = []
value = []
with open('df_1Week_Dunne.csv') as f:
    cf = csv.DictReader(f, fieldnames=['time_stamp', 'value'])
    for row in cf:
        time_stamp.append(row['time_stamp'])
        value.append(float(row['value']))


In [63]:
time.strftime("%Y-%m-%d",  time.strptime(time_stamp[0], "%Y-%m-%d %H:%M:%S"))

'2015-09-19'

In [42]:
for items in df_1Week:
    print items['time_stamp']

2015-09-19 00:30:00
2015-09-19 01:00:00
2015-09-19 01:30:00
2015-09-19 02:00:00
2015-09-19 02:30:00
2015-09-19 03:00:00
2015-09-19 03:30:00
2015-09-19 04:00:00
2015-09-19 04:30:00
2015-09-19 05:00:00
2015-09-19 05:30:00
2015-09-19 06:00:00
2015-09-19 06:30:00
2015-09-19 07:00:00
2015-09-19 07:30:00
2015-09-19 08:00:00
2015-09-19 08:30:00
2015-09-19 09:00:00
2015-09-19 09:30:00
2015-09-19 10:00:00
2015-09-19 10:30:00
2015-09-19 11:00:00
2015-09-19 11:30:00
2015-09-19 12:00:00
2015-09-19 12:30:00
2015-09-19 13:00:00
2015-09-19 13:30:00
2015-09-19 14:00:00
2015-09-19 14:30:00
2015-09-19 15:00:00
2015-09-19 15:30:00
2015-09-19 16:00:00
2015-09-19 16:30:00
2015-09-19 17:00:00
2015-09-19 17:30:00
2015-09-19 18:00:00
2015-09-19 18:30:00
2015-09-19 19:00:00
2015-09-19 19:30:00
2015-09-19 20:00:00
2015-09-19 20:30:00
2015-09-19 21:00:00
2015-09-19 21:30:00
2015-09-19 22:00:00
2015-09-19 22:30:00
2015-09-19 23:00:00
2015-09-19 23:30:00
2015-09-20 00:00:00
2015-09-20 00:30:00
2015-09-20 01:00:00
